In [92]:
import os 
import cv2
import numpy as np
import tensorflow
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [68]:
from keras.preprocessing.image import ImageDataGenerator

In [69]:
datapath = "./data"
outputmodel = "./videoClassificationModel"
outputlabelninarizer = "./videoClassificationAndBinarizer"
epoch=25

In [70]:
Sports_Labels = set(['boxing','swimming','table_tennis']) 
print("Images is being loaded")
pathToImages = list(paths.list_images(datapath))
data = []
labels = []

Images is being loaded


In [71]:
for images in pathToImages:
    label = images.split(os.path.sep)[-2]
    if label not in Sports_Labels:
        continue
    image = cv2.imread(images)
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (244,244))
    data.append(image) 
    labels.append(label)

libpng warning: iCCP: known incorrect sRGB profile


In [72]:
data = np.array(data)
labels = np.array(labels)
#hot encoded values as 0,1,2
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [73]:
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels, test_size=0.25, stratify=labels, random_state=42)

In [74]:
traininAugmentatioin = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [75]:
validationAugmentation = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
traininAugmentatioin.mean = mean
validationAugmentation.mean = mean

In [76]:
from keras.layers import Input
from keras.layers import AveragePooling2D
from keras.applications import ResNet50
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Model

In [96]:
baseModel = ResNet50(weights="imagenet", include_top= False, input_tensor=Input(shape=(224,244,3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7)) (headModel)
headModel = Flatten (name="flatten")(headModel)
headModel = Dense(512, activation="relu") (headModel)
headModel = Dropout (0.5) (headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model (inputs=baseModel.input , outputs=headModel)

for basemodelLayers in baseModel.layers:
    basemodelLayers. trainable = False

In [97]:
from keras.optimizers.legacy import SGD

In [98]:
opt = SGD(learning_rate=0.0001,momentum=0.9, decay = 1e-4/epoch)

In [99]:
model. compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [100]:
History = model.fit(
traininAugmentatioin.flow(X_train, Y_train, batch_size=32),
steps_per_epoch=len(X_train) // 32,
validation_data=validationAugmentation.flow(X_test,Y_test),
validation_steps=len(X_test) // 32, epochs=epoch
)

Epoch 1/25
49/49 [==============================] - 84s 2s/step - loss: 1.3285 - accuracy: 0.4541 - val_loss: 0.5700 - val_accuracy: 0.7793
Epoch 2/25
49/49 [==============================] - 88s 2s/step - loss: 0.6867 - accuracy: 0.7033 - val_loss: 0.3597 - val_accuracy: 0.8828
Epoch 3/25
49/49 [==============================] - 114s 2s/step - loss: 0.5123 - accuracy: 0.7899 - val_loss: 0.2938 - val_accuracy: 0.8984
Epoch 4/25
49/49 [==============================] - 90s 2s/step - loss: 0.4724 - accuracy: 0.8107 - val_loss: 0.2422 - val_accuracy: 0.9258
Epoch 5/25
49/49 [==============================] - 91s 2s/step - loss: 0.3898 - accuracy: 0.8412 - val_loss: 0.2175 - val_accuracy: 0.9258
Epoch 6/25
49/49 [==============================] - 89s 2s/step - loss: 0.3612 - accuracy: 0.8575 - val_loss: 0.1936 - val_accuracy: 0.9375
Epoch 7/25
49/49 [==============================] - 91s 2s/step - loss: 0.3462 - accuracy: 0.8712 - val_loss: 0.1847 - val_accuracy: 0.9355
Epoch 8/25
49/49 [=

In [101]:
import pickle
model.save(outputmodel)
lbinarizer = open("./videoClassificationAndBinarizer.pickle","wb")
lbinarizer.write(pickle.dumps(lb))
lbinarizer.close()

INFO:tensorflow:Assets written to: ./videoClassificationModel/assets


INFO:tensorflow:Assets written to: ./videoClassificationModel/assets
